

### **Authenticate & Install BigQuery**

In [1]:
# Authenticate Google Cloud access
from google.colab import auth
auth.authenticate_user()

# Install required package
!pip install --quiet --upgrade google-cloud-bigquery


### Import Libraries

In [2]:
import requests
import pandas as pd
from datetime import datetime
from google.cloud import bigquery


### Get Top 100 BVG Stops (from Location API)

In [3]:
# Function to get 100 top busy stops from BVG
def get_top_stops(query="Berlin", limit=100):
    url = f"https://v6.bvg.transport.rest/locations?query={query}&results={limit}"
    response = requests.get(url)
    data = response.json()

    stops = []
    for item in data:
        if item['type'] == 'stop':
            stops.append({
                'stop_id': item['id'],
                'stop_name': item['name']
            })

    return pd.DataFrame(stops)

# Load top 100 stops
stop_df = get_top_stops()
print(f"✅ Loaded {len(stop_df)} stops")
stop_df.head()


✅ Loaded 100 stops


,stop_id,stop_name
0,900037168,"Berlin, Albrechtshof Bhf"
1,900029305,"Berlin, Staaken Bhf"
2,900180008,Bahnhofstr./Lindenstr. (Berlin)
3,900180009,Bahnhofstr./Seelenbinderstr. (Berlin)
4,900170011,Barnimplatz (Berlin)


### Collect Real-Time Departures from Top 100 Stops

In [4]:
# Fetch real-time departures from all 100 stops
all_data = []

for i, row in stop_df.iterrows():
    stop_id = row['stop_id']
    stop_name = row['stop_name']
    url = f"https://v6.bvg.transport.rest/stops/{stop_id}/departures"
    params = {"duration": 60}

    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        for item in data.get('departures', []):
            all_data.append({
                "timestamp_collected": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "stop_name": stop_name,
                "stop_id": stop_id,
                "arrival_time": item.get("when"),
                "direction": item.get("direction"),
                "line": item.get("line", {}).get("name"),
                "delay_seconds": item.get("delay"),
                "platform": item.get("platform")
            })
    else:
        print(f"❌ Failed to fetch {stop_name}: {response.status_code}")

print(f"✅ Total records collected: {len(all_data)}")


✅ Total records collected: 1463


### Clean and Transform the Data

In [9]:
# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(all_data)

# Show the first few rows
print("📊 Sample of collected data:")
df.head()

📊 Sample of collected data:


,timestamp_collected,stop_name,stop_id,arrival_time,direction,line,delay_seconds,platform
0,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07T21:43:00+02:00,S+U Rathaus Spandau,237,0.0,None
1,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07T22:03:00+02:00,S+U Rathaus Spandau,237,NaN,None
2,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07T22:07:00+02:00,"Nauen, Bahnhof",RE2,NaN,None
3,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07T22:23:00+02:00,S+U Rathaus Spandau,237,0.0,None
4,2025-04-07 19:30:24,"Berlin, Staaken Bhf",900029305,2025-04-07T21:30:00+02:00,"Staaken, Heidebergplan",M32,60.0,None


In [10]:

if df.empty:
    print("⚠️ No data collected.")
else:
    df['arrival_time'] = pd.to_datetime(df['arrival_time'])
    df['timestamp_collected'] = pd.to_datetime(df['timestamp_collected'])
    df['delay_minutes'] = df['delay_seconds'] / 60
    df['arrival_hour'] = df['arrival_time'].dt.hour
    df['arrival_day'] = df['arrival_time'].dt.date
    df['arrival_weekday'] = df['arrival_time'].dt.day_name()
    df = df.fillna({
        'direction': 'Unknown',
        'line': 'Unknown',
        'platform': 'Unknown',
        'delay_minutes': 0
    })

    df.head()

In [11]:
df.head()

,timestamp_collected,stop_name,stop_id,arrival_time,direction,line,delay_seconds,platform,delay_minutes,arrival_hour,arrival_day,arrival_weekday
0,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07 21:43:00+02:00,S+U Rathaus Spandau,237,0.0,Unknown,0.0,21,2025-04-07,Monday
1,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07 22:03:00+02:00,S+U Rathaus Spandau,237,NaN,Unknown,0.0,22,2025-04-07,Monday
2,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07 22:07:00+02:00,"Nauen, Bahnhof",RE2,NaN,Unknown,0.0,22,2025-04-07,Monday
3,2025-04-07 19:30:24,"Berlin, Albrechtshof Bhf",900037168,2025-04-07 22:23:00+02:00,S+U Rathaus Spandau,237,0.0,Unknown,0.0,22,2025-04-07,Monday
4,2025-04-07 19:30:24,"Berlin, Staaken Bhf",900029305,2025-04-07 21:30:00+02:00,"Staaken, Heidebergplan",M32,60.0,Unknown,1.0,21,2025-04-07,Monday


### Upload to BigQuery

In [13]:
from pandas_gbq import to_gbq

to_gbq(
    dataframe=df,
    destination_table="transport_data.departures",
    project_id="bvg-transport-project-455913",
    if_exists="append"
)


100%|██████████| 1/1 [00:00<00:00, 8648.05it/s]


### Save to CSV

In [14]:
# Save locally in Colab and download
filename = f"bvg_departures_top100_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
df.to_csv(filename, index=False)

from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>